# Phase 2: Kernel Fusion + CUDA Streams Benchmark

This notebook tests Phase 2 optimizations:
1. **Fused RMSNorm+MatMul kernel** - Combines normalization and matrix multiplication
2. **CUDA Streams** - Async H2D/D2H transfers
3. **Pinned Memory** - Page-locked host memory for faster transfers

**Target**: 400+ tok/s (closing 1.4x gap with Ollama's 423 tok/s)

**Requirements:**
- NVIDIA GPU (T4, RTX, Jetson)
- CUDA Toolkit 11.0+

## 1. Environment Setup

In [ ]:
# Check GPU
!nvidia-smi --query-gpu=name,memory.total,compute_cap --format=csv

In [ ]:
# Clone repository
import os
if not os.path.exists('ollama-api-gateway'):
    !git clone https://github.com/umerkhan95/ollama-api-gateway.git
else:
    print('Repository exists, pulling latest...')
    !cd ollama-api-gateway && git pull

In [ ]:
# Build CUDA kernels
%cd ollama-api-gateway/mojo-gateway/src/kernels
!make cuda
!ls -la ../../lib/

## 2. Load CUDA Library with Phase 2 APIs

In [ ]:
import ctypes
import numpy as np
import time

# Load library
lib_path = '../../lib/libtmac_kernel_cuda.so'
cuda_lib = ctypes.CDLL(lib_path)
print(f'Loaded: {lib_path}')

# Define function signatures
# Basic functions
cuda_lib.cuda_available.restype = ctypes.c_int
cuda_lib.cuda_device_name.restype = ctypes.c_char_p
cuda_lib.cuda_init.argtypes = [ctypes.c_int, ctypes.c_int, ctypes.c_int]
cuda_lib.cuda_init.restype = ctypes.c_int
cuda_lib.cuda_cleanup.restype = None
cuda_lib.cuda_sync.restype = None

# Phase 1: Persistent memory API
cuda_lib.cuda_load_weights.argtypes = [
    ctypes.POINTER(ctypes.c_int8),  # weights
    ctypes.POINTER(ctypes.c_float),  # scales
    ctypes.c_int,  # weight_bytes
    ctypes.c_int   # num_rows
]
cuda_lib.cuda_load_weights.restype = ctypes.c_int
cuda_lib.cuda_unload_weights.restype = None
cuda_lib.cuda_weights_loaded.restype = ctypes.c_int

cuda_lib.tmac_matmul_cuda_persistent.argtypes = [
    ctypes.POINTER(ctypes.c_float),  # activations
    ctypes.POINTER(ctypes.c_float),  # output
    ctypes.c_int, ctypes.c_int, ctypes.c_int  # M, N, K
]
cuda_lib.tmac_matmul_cuda_persistent.restype = ctypes.c_int

cuda_lib.cuda_load_norm_weights.argtypes = [
    ctypes.POINTER(ctypes.c_float),  # weights
    ctypes.c_int  # size
]
cuda_lib.cuda_load_norm_weights.restype = ctypes.c_int

cuda_lib.rmsnorm_cuda_persistent.argtypes = [
    ctypes.POINTER(ctypes.c_float),  # output
    ctypes.POINTER(ctypes.c_float),  # input
    ctypes.c_int, ctypes.c_int, ctypes.c_float  # batch_size, size, eps
]
cuda_lib.rmsnorm_cuda_persistent.restype = ctypes.c_int

# Phase 2: Kernel Fusion + Streams API
cuda_lib.cuda_init_streams.restype = ctypes.c_int
cuda_lib.cuda_cleanup_streams.restype = None
cuda_lib.cuda_sync_streams.restype = None

cuda_lib.cuda_alloc_pinned.argtypes = [ctypes.c_int, ctypes.c_int]
cuda_lib.cuda_alloc_pinned.restype = ctypes.c_int
cuda_lib.cuda_free_pinned.restype = None

# Fused RMSNorm + MatMul
cuda_lib.fused_rmsnorm_matmul_cuda.argtypes = [
    ctypes.POINTER(ctypes.c_float),  # input
    ctypes.POINTER(ctypes.c_float),  # output
    ctypes.c_int, ctypes.c_int, ctypes.c_int,  # M, N, K
    ctypes.c_float  # eps
]
cuda_lib.fused_rmsnorm_matmul_cuda.restype = ctypes.c_int

# Async MatMul
cuda_lib.tmac_matmul_cuda_async.argtypes = [
    ctypes.POINTER(ctypes.c_float),  # activations
    ctypes.POINTER(ctypes.c_float),  # output
    ctypes.c_int, ctypes.c_int, ctypes.c_int  # M, N, K
]
cuda_lib.tmac_matmul_cuda_async.restype = ctypes.c_int

# Fast fused kernel with all optimizations
cuda_lib.fused_rmsnorm_matmul_cuda_fast.argtypes = [
    ctypes.POINTER(ctypes.c_float),  # input
    ctypes.POINTER(ctypes.c_float),  # output
    ctypes.c_int, ctypes.c_int, ctypes.c_int,  # M, N, K
    ctypes.c_float  # eps
]
cuda_lib.fused_rmsnorm_matmul_cuda_fast.restype = ctypes.c_int

print('Function signatures defined')

In [ ]:
# Check CUDA and initialize
if cuda_lib.cuda_available():
    device_name = cuda_lib.cuda_device_name().decode('utf-8')
    print(f'CUDA Device: {device_name}')
else:
    raise RuntimeError('No CUDA device found!')

# Initialize with generous buffer sizes
max_weights = 100_000_000  # 100MB for weights
max_activations = 10_000_000
max_output = 10_000_000
ret = cuda_lib.cuda_init(max_weights, max_activations, max_output)
if ret == 0:
    print('CUDA initialized successfully')
else:
    raise RuntimeError('CUDA initialization failed')

# Initialize streams
ret = cuda_lib.cuda_init_streams()
if ret == 0:
    print('CUDA streams initialized')

# Allocate pinned memory
ret = cuda_lib.cuda_alloc_pinned(max_activations, max_output)
if ret == 0:
    print('Pinned memory allocated')

## 3. Model Parameters (SmolLM-135M)

In [ ]:
# SmolLM-135M architecture
hidden_size = 576
intermediate_size = 1536
num_heads = 9
head_dim = hidden_size // num_heads  # 64
vocab_size = 49152
num_layers = 9

def calc_weight_bytes(out_features, in_features):
    return out_features * ((in_features + 3) // 4)

print(f'Hidden size: {hidden_size}')
print(f'Intermediate size: {intermediate_size}')
print(f'Num layers: {num_layers}')

## 4. Benchmark: Phase 1 vs Phase 2 APIs

In [ ]:
def benchmark_phase1_separate(weights, activations, output, scales, norm_weights, M, N, K, iterations=100):
    """Phase 1: Separate RMSNorm + MatMul calls."""
    weights_ptr = weights.ctypes.data_as(ctypes.POINTER(ctypes.c_int8))
    act_ptr = activations.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    out_ptr = output.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    scales_ptr = scales.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    norm_ptr = norm_weights.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    
    weight_bytes = M * ((K + 3) // 4)
    norm_out = np.zeros(K, dtype=np.float32)
    norm_out_ptr = norm_out.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    
    # Load weights once
    cuda_lib.cuda_load_weights(weights_ptr, scales_ptr, weight_bytes, M)
    cuda_lib.cuda_load_norm_weights(norm_ptr, K)
    
    # Warmup
    for _ in range(10):
        cuda_lib.rmsnorm_cuda_persistent(norm_out_ptr, act_ptr, 1, K, ctypes.c_float(1e-6))
        cuda_lib.tmac_matmul_cuda_persistent(norm_out_ptr, out_ptr, M, N, K)
    cuda_lib.cuda_sync()
    
    # Benchmark
    start = time.perf_counter()
    for _ in range(iterations):
        cuda_lib.rmsnorm_cuda_persistent(norm_out_ptr, act_ptr, 1, K, ctypes.c_float(1e-6))
        cuda_lib.tmac_matmul_cuda_persistent(norm_out_ptr, out_ptr, M, N, K)
    cuda_lib.cuda_sync()
    end = time.perf_counter()
    
    cuda_lib.cuda_unload_weights()
    return (end - start) / iterations * 1000  # ms per call

def benchmark_phase2_fused(weights, activations, output, scales, norm_weights, M, N, K, iterations=100):
    """Phase 2: Fused RMSNorm + MatMul kernel."""
    weights_ptr = weights.ctypes.data_as(ctypes.POINTER(ctypes.c_int8))
    act_ptr = activations.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    out_ptr = output.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    scales_ptr = scales.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    norm_ptr = norm_weights.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    
    weight_bytes = M * ((K + 3) // 4)
    
    # Load weights once
    cuda_lib.cuda_load_weights(weights_ptr, scales_ptr, weight_bytes, M)
    cuda_lib.cuda_load_norm_weights(norm_ptr, K)
    
    # Warmup
    for _ in range(10):
        cuda_lib.fused_rmsnorm_matmul_cuda(act_ptr, out_ptr, M, N, K, ctypes.c_float(1e-6))
    cuda_lib.cuda_sync()
    
    # Benchmark
    start = time.perf_counter()
    for _ in range(iterations):
        cuda_lib.fused_rmsnorm_matmul_cuda(act_ptr, out_ptr, M, N, K, ctypes.c_float(1e-6))
    cuda_lib.cuda_sync()
    end = time.perf_counter()
    
    cuda_lib.cuda_unload_weights()
    return (end - start) / iterations * 1000  # ms per call

def benchmark_phase2_fast(weights, activations, output, scales, norm_weights, M, N, K, iterations=100):
    """Phase 2: Fast fused kernel with streams + pinned memory."""
    weights_ptr = weights.ctypes.data_as(ctypes.POINTER(ctypes.c_int8))
    act_ptr = activations.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    out_ptr = output.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    scales_ptr = scales.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    norm_ptr = norm_weights.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    
    weight_bytes = M * ((K + 3) // 4)
    
    # Load weights once
    cuda_lib.cuda_load_weights(weights_ptr, scales_ptr, weight_bytes, M)
    cuda_lib.cuda_load_norm_weights(norm_ptr, K)
    
    # Warmup
    for _ in range(10):
        cuda_lib.fused_rmsnorm_matmul_cuda_fast(act_ptr, out_ptr, M, N, K, ctypes.c_float(1e-6))
    cuda_lib.cuda_sync_streams()
    
    # Benchmark
    start = time.perf_counter()
    for _ in range(iterations):
        cuda_lib.fused_rmsnorm_matmul_cuda_fast(act_ptr, out_ptr, M, N, K, ctypes.c_float(1e-6))
    cuda_lib.cuda_sync_streams()
    end = time.perf_counter()
    
    cuda_lib.cuda_unload_weights()
    return (end - start) / iterations * 1000  # ms per call

In [ ]:
# Test configurations simulating transformer layers
test_configs = [
    ('QKV Projection', 3 * hidden_size, 1, hidden_size),
    ('Output Projection', hidden_size, 1, hidden_size),
    ('FFN Up', intermediate_size, 1, hidden_size),
    ('FFN Down', hidden_size, 1, intermediate_size),
]

print('=' * 100)
print('PHASE 2 BENCHMARK: Kernel Fusion + CUDA Streams')
print('=' * 100)
print(f'{"Layer":<20} {"M":<8} {"K":<8} {"Phase1 (ms)":<15} {"Fused (ms)":<15} {"Fast (ms)":<15} {"Speedup":<10}')
print('-' * 100)

results = []
iterations = 100

for name, M, N, K in test_configs:
    # Create test data
    weight_bytes = M * ((K + 3) // 4)
    weights = np.random.randint(-1, 2, size=weight_bytes, dtype=np.int8)
    activations = np.random.randn(K * N).astype(np.float32)
    output = np.zeros(M * N, dtype=np.float32)
    scales = np.ones(M, dtype=np.float32)
    norm_weights = np.ones(K, dtype=np.float32)
    
    try:
        # Benchmark all three approaches
        phase1_ms = benchmark_phase1_separate(weights, activations, output, scales, norm_weights, M, N, K, iterations)
        fused_ms = benchmark_phase2_fused(weights, activations, output, scales, norm_weights, M, N, K, iterations)
        fast_ms = benchmark_phase2_fast(weights, activations, output, scales, norm_weights, M, N, K, iterations)
        
        speedup = phase1_ms / fast_ms
        print(f'{name:<20} {M:<8} {K:<8} {phase1_ms:<15.3f} {fused_ms:<15.3f} {fast_ms:<15.3f} {speedup:<10.2f}x')
        results.append((name, M, K, phase1_ms, fused_ms, fast_ms, speedup))
    except Exception as e:
        print(f'{name:<20} ERROR: {e}')

print('-' * 100)

if results:
    avg_speedup = sum(r[6] for r in results) / len(results)
    print(f'\nAverage Speedup (Phase 1 → Phase 2 Fast): {avg_speedup:.2f}x')

## 5. Transformer Layer Simulation

In [ ]:
def simulate_layer_phase1(iterations=100):
    """Simulate one layer with Phase 1 (separate kernels)."""
    # Create weights for QKV and FFN
    qkv_weight_bytes = (3 * hidden_size) * ((hidden_size + 3) // 4)
    qkv_weights = np.random.randint(-1, 2, size=qkv_weight_bytes, dtype=np.int8)
    qkv_scales = np.ones(3 * hidden_size, dtype=np.float32)
    
    ffn_up_bytes = intermediate_size * ((hidden_size + 3) // 4)
    ffn_up_weights = np.random.randint(-1, 2, size=ffn_up_bytes, dtype=np.int8)
    ffn_up_scales = np.ones(intermediate_size, dtype=np.float32)
    
    hidden = np.random.randn(hidden_size).astype(np.float32)
    norm_weights = np.ones(hidden_size, dtype=np.float32)
    qkv_output = np.zeros(3 * hidden_size, dtype=np.float32)
    ffn_output = np.zeros(intermediate_size, dtype=np.float32)
    norm_out = np.zeros(hidden_size, dtype=np.float32)
    
    # Pointers
    hidden_ptr = hidden.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    norm_ptr = norm_weights.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    norm_out_ptr = norm_out.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    qkv_w_ptr = qkv_weights.ctypes.data_as(ctypes.POINTER(ctypes.c_int8))
    qkv_s_ptr = qkv_scales.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    qkv_o_ptr = qkv_output.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    ffn_w_ptr = ffn_up_weights.ctypes.data_as(ctypes.POINTER(ctypes.c_int8))
    ffn_s_ptr = ffn_up_scales.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    ffn_o_ptr = ffn_output.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    
    # Load QKV weights
    cuda_lib.cuda_load_weights(qkv_w_ptr, qkv_s_ptr, qkv_weight_bytes, 3 * hidden_size)
    cuda_lib.cuda_load_norm_weights(norm_ptr, hidden_size)
    
    # Warmup
    for _ in range(5):
        cuda_lib.rmsnorm_cuda_persistent(norm_out_ptr, hidden_ptr, 1, hidden_size, ctypes.c_float(1e-6))
        cuda_lib.tmac_matmul_cuda_persistent(norm_out_ptr, qkv_o_ptr, 3*hidden_size, 1, hidden_size)
    cuda_lib.cuda_sync()
    
    # Benchmark (QKV projection only for fair comparison)
    start = time.perf_counter()
    for _ in range(iterations):
        cuda_lib.rmsnorm_cuda_persistent(norm_out_ptr, hidden_ptr, 1, hidden_size, ctypes.c_float(1e-6))
        cuda_lib.tmac_matmul_cuda_persistent(norm_out_ptr, qkv_o_ptr, 3*hidden_size, 1, hidden_size)
    cuda_lib.cuda_sync()
    end = time.perf_counter()
    
    cuda_lib.cuda_unload_weights()
    return (end - start) / iterations * 1000

def simulate_layer_phase2(iterations=100):
    """Simulate one layer with Phase 2 (fused kernel)."""
    qkv_weight_bytes = (3 * hidden_size) * ((hidden_size + 3) // 4)
    qkv_weights = np.random.randint(-1, 2, size=qkv_weight_bytes, dtype=np.int8)
    qkv_scales = np.ones(3 * hidden_size, dtype=np.float32)
    
    hidden = np.random.randn(hidden_size).astype(np.float32)
    norm_weights = np.ones(hidden_size, dtype=np.float32)
    qkv_output = np.zeros(3 * hidden_size, dtype=np.float32)
    
    hidden_ptr = hidden.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    norm_ptr = norm_weights.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    qkv_w_ptr = qkv_weights.ctypes.data_as(ctypes.POINTER(ctypes.c_int8))
    qkv_s_ptr = qkv_scales.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    qkv_o_ptr = qkv_output.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    
    # Load weights
    cuda_lib.cuda_load_weights(qkv_w_ptr, qkv_s_ptr, qkv_weight_bytes, 3 * hidden_size)
    cuda_lib.cuda_load_norm_weights(norm_ptr, hidden_size)
    
    # Warmup
    for _ in range(5):
        cuda_lib.fused_rmsnorm_matmul_cuda_fast(hidden_ptr, qkv_o_ptr, 3*hidden_size, 1, hidden_size, ctypes.c_float(1e-6))
    cuda_lib.cuda_sync_streams()
    
    # Benchmark
    start = time.perf_counter()
    for _ in range(iterations):
        cuda_lib.fused_rmsnorm_matmul_cuda_fast(hidden_ptr, qkv_o_ptr, 3*hidden_size, 1, hidden_size, ctypes.c_float(1e-6))
    cuda_lib.cuda_sync_streams()
    end = time.perf_counter()
    
    cuda_lib.cuda_unload_weights()
    return (end - start) / iterations * 1000

In [ ]:
print('\n' + '=' * 60)
print('TRANSFORMER LAYER SIMULATION (QKV Projection)')
print('=' * 60)

phase1_ms = simulate_layer_phase1(iterations=100)
phase2_ms = simulate_layer_phase2(iterations=100)

print(f'\nPhase 1 (separate kernels): {phase1_ms:.3f} ms')
print(f'Phase 2 (fused + streams):  {phase2_ms:.3f} ms')
print(f'Speedup: {phase1_ms / phase2_ms:.2f}x')

# Estimate full model throughput
print('\n' + '=' * 60)
print('ESTIMATED FULL MODEL THROUGHPUT (SmolLM-135M, 9 layers)')
print('=' * 60)

# Each layer has multiple operations; estimate based on QKV projection being ~30% of layer time
layer_ops_factor = 3  # QKV + O_proj + FFN
phase1_layer_ms = phase1_ms * layer_ops_factor
phase2_layer_ms = phase2_ms * layer_ops_factor

phase1_token_ms = phase1_layer_ms * num_layers
phase2_token_ms = phase2_layer_ms * num_layers

phase1_tok_s = 1000 / phase1_token_ms
phase2_tok_s = 1000 / phase2_token_ms

print(f'\nPhase 1 (Persistent Memory):')
print(f'  Per token: {phase1_token_ms:.1f} ms')
print(f'  Throughput: {phase1_tok_s:.1f} tok/s')

print(f'\nPhase 2 (Kernel Fusion + Streams):')
print(f'  Per token: {phase2_token_ms:.1f} ms')
print(f'  Throughput: {phase2_tok_s:.1f} tok/s')

print(f'\nImprovement: {phase2_tok_s / phase1_tok_s:.2f}x faster than Phase 1')
print(f'\nOllama comparison: {phase2_tok_s:.1f} tok/s vs 423 tok/s ({phase2_tok_s/423*100:.1f}% of Ollama)')

## 6. Cleanup

In [ ]:
cuda_lib.cuda_free_pinned()
cuda_lib.cuda_cleanup_streams()
cuda_lib.cuda_cleanup()
print('CUDA resources cleaned up')

## 7. Summary

**Phase 2 Optimizations:**
- Fused RMSNorm + MatMul kernel (50% fewer kernel launches)
- CUDA streams for async H2D/D2H transfers
- Pinned memory for faster PCIe transfers

**Expected Results:**
- Phase 1: ~296 tok/s (established baseline)
- Phase 2: Target 400+ tok/s
- Ollama: 423 tok/s

**Next Steps:**
- Phase 3: T-MAC tensor core optimization (INT8 WMMA)
- Phase 3: CUDA Graphs for full forward pass
- Phase 3: Warp-private LUTs (remove atomicAdd)